<a href="https://www.kaggle.com/code/dplayer/task-2-rentalpricedata-exploration?scriptVersionId=99808231" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/June2022-June2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Oct2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Nov2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Feb2022-Feb2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Apr2022-Apr2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Sep2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Dec2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Mar2022-Mar2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Jan2022-Jan2022.csv
/kaggle/input/housing-affordability-in-canada/Structural-dwellings-household-size.csv
/kaggle/input/housing-affordability-in-canada/population_dwellings_count.csv

Load data

Clean the prices column

In [6]:
## imports
from pathlib import Path

In [3]:
import re
p = re.compile(r'\D')

In [18]:
def cleanupandadddate(file_name, month, year):
    df = pd.read_csv(Path(file_name))
    print(df)
    try:
        df['1 BED'] = [p.sub('', x) for x in df['1 BED']]
    except TypeError as te:
        print(f"1-BED {te}")
    try:
        df['2 BED'] = [p.sub('', x) for x in df['2 BED']]
    except TypeError as te:
        print(f"2-BED {te}")
    try:
        df['M/M'] = [re.sub(r'%', '', str(x)) for x in df['M/M']]
    except TypeError as te:
        print(f"M/M {te}")
    try:
        df['Y/Y'] = [re.sub(r'%', '', str(x)) for x in df['Y/Y']]
    except TypeError as te:
        print(f"Y/Y {te}")
    try:
        df['M/M.1'] = [re.sub(r'%', '', str(x)) for x in df['M/M.1']]
    except TypeError as te:
        print(f"M/M.1 {te}")
    try:
        df['Y/Y.1'] = [re.sub(r'%', '', str(x)) for x in df['Y/Y.1']]
    except TypeError as te:
        print(f"Y/Y.1 {te}")
    df['Month'] =[month]*df.shape[0]
    df['Year'] = [year]*df.shape[0]
    df.columns = ['City', '1-Bed($)', 'M/M-1-BED(%)', 'Y/Y-1-BED(%)', '2-BED', 'M/M-2-BED(%)', 'Y/Y-2-BED(%)', 'Month', 'Year']
    return df

In [7]:
updfMay = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv', 'May', '2022')
print(updfMay)

              City   1 BED     M/M    Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,334   2.37%  18.12  $3,324   6.47%  23.71
1          Toronto  $2,065   2.08%  12.96  $2,849   2.63%  17.19
2       Burlington  $2,017   0.00%  15.85  $2,261   0.22%  11.76
3         Oakville  $2,010   5.29%   8.12  $2.333   0.09%   3.37
4        Etobicoke  $1,897   2.49%  10.68  $2.446   4.80%  15.87
5          Burnaby  $1,883  -3.88%  15.24  $2,563   5.39%  14.52
6      Mississauga  $1,817   0.11%   5.27  $2,211   0.32%   7.64
7           Guelph  $1,811   3.37%  12.91  $2,179  -0.23%  16.09
8      Victoria BC  $1.776   0.74%   6.86  $2,736   7.67%    NaN
9           Barrie  $1.771   0.11%   2.02  $2.084   0.82%  -3.43
10         Vaughan  $1,730   6.86%  11.54  $2.338   6.08%  10.44
11      North York  $1,725    1.29   4.10  $2,259   3.62%   8.82
12       Kitchener  $1,701   5.59%  11.39  $2,037  -1.69%  17.34
13      Surrev. BC  $1.655   6.64%  12.89  $2,080    5.96    NaN
14          Oshawa  $1,64

In [8]:
updfApr = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Apr2022-Apr2022.csv', 'April', '2022')
print(updfApr)

              City   1 BED     M/M    Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,280   1.83%  13.94  $3.122   2.36%  17.59
1          Toronto  $2,023  -1.03%  10.97  $2,776  -0.07%  16.15
2       Burlington  $2,017   8.67%  12.74  $2,256  -1.01%  12.46
3          Burnaby  $1.959  -1.01%  16.47  $2,432   1.42%   9.85
4         Oakville  $1,909  -0.68%   8.71  $2.331  -3.72%  10.95
5        Etobicoke  $1,851  -0.48%   8.18  $2,334  -4.81%   9.42
6      Mississauga  $1,815   2.20%   4.01  $2,204   0.50%   7.83
7           Barrie   $1769  -0.11%   3.21  $2,067   1.87%   7.99
8         Victoria  $1.763   6.14%   7.63  $2,541  -4.72%    NaN
9           Guelph  $1.752   1.33%   9.30   $2184   3.41%  11.20
10          Oshawa  $1.657  -9.06%  18.70  $1.906   3.25%   1.17
11            Aiax  $1,650   2.61%   2.42  $1,908  -1.29%  -3.10
12         Halifax  $1,641  -1.32%  17.33  $2,064   1.78%  16.41
13          Ottawa  $1.640  -3.07%   5.94  $2.030  -0.15%   2.16
14       Brantford  $1,61

In [9]:
df = pd.read_csv(Path('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    35 non-null     object
 1   1 BED   35 non-null     object
 2   M/M     35 non-null     object
 3   Y/Y     34 non-null     object
 4   2 BED   35 non-null     int64 
 5   M/M.1   35 non-null     object
 6   Y/Y.1   34 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.0+ KB


In [10]:
updfAug = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv', 'August', '2021')
print(updfAug)
updfAug.info()

               City   1 BED     M/M     Y/Y  2 BED  M/M.1   Y/Y.1
0     Vancouver. BC   $2185    6.3%   14.4%   3041     10    14.5
1           Toronto  $1,855    1.0%   -9.0%   2606   4.2%   -3.2%
2         Etobicoke  $1,802    2.8%   -7.2%   2361   3.6%   -4.3%
3          Victoria  $1,756    8.7%   11.6%   2103   2.0%    4.6%
4       Mississauga  $1,753   -0.3%   -9.3%   2129   1.6%   -2.4%
5        Burlington  $1,745   -2.2%  -10.7%   2065  -0.8%    1.7%
6            Guelph  $1.714    2.2%   10.2%   2000   2.1%    8.9%
7           Vaughan  $1,699    9.2%   -9.2%   2115  -2.3%   -6.6%
8           Burnaby  $1,672    0.2%   -3.7%   2244   0.1%    4.4%
9        North York  $1,669   -0.1%   -9.7%   2025  -0.3%  -10.0%
10  New Westminster  $1.668    4.8%   10.9%   2010  -5.7%   -6.8%
11           Ottawa  $1.611    1.3%   -3.4%   2033   3.2%    -6.1
12      Scarborough  $1,586    0.2%    -5.4   1852  -1.9%   -8.2%
13          Halifax  $1,576    2.9%     NaN   2033   5.7%     NaN
14        

In [11]:
updfDec = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Dec2021-Table 1.csv', 'December', '2021')
print(updfDec)

               City  1 BED    M/M      Y/Y   2 BED   M/M.1    Y/Y.1
0         Vancouver   2132  -3.79    7.73%  $2,998   -1.93   10.71%
1           Toronto   2040   1.69    7.99%  $2,764   3.25%   11.09%
2          Oakville   1879  -5.29    -1.78  $2,322  -1.02%    3.38%
3        Burlington   1827  -3.23     4.09  $2,275   1.38%   11.90%
4         Etobicoke   1809   2.26   -0.39%  $2.302  -0.04%    1.81%
5       Mississauga   1796  -1.54   -2.55%  $2,142  -0.83%    0.33%
6            Barrie   1746 -12.87   14.27%  $1,989  -8.93%    -1.53
7           Markham   1715  -0.52    1.96%  $1,972  -1.55%   -2.28%
8            Guelph   1704  -3.51   12.25%  $2,118   0.14%   12.84%
9            Oshawa   1701    NaN   13.25%  $1.783   -3.83    5.94%
10          Burnaby   1696  -3.58   -0.47%  $2,414   5.88%    9.83%
11         Victoria   1695  -9.02    8.10%  $2,563   5.39%      NaN
12       North York   1659  -3.77    -9.2%  $2.120   0.43%  -12.97%
13          Halifax   1652   0.92    7.34%  $1,9

In [12]:
updfFeb = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Feb2022-Feb2022.csv', 'February', '2022')
print(updfFeb)

              City  1 BED      M/M     Y/Y  2 BED   M/M.1  Y/Y.1
0        Vancouver   2163   -0.60%    9.24   3003   0.67%  12.30
1          Toronto   2027    0.70%   11.50   2769   1.99%  14.56
2         Oakville   1959    2.83%    9.69   2388  -0.50%  12.01
3        Etobicoke   1857    0.00%    5.09   2424    0.66  10.13
4       Burlington   1851    0.71%    0.00   2317  -0.17%  12.59
5           Oshawa   1849    -3.55  20.93%   1739  -1.86%   0.06
6          Burnabv   1847   -7.23%    7.38   2377   -0.79   7.80
7    Richmond Hill   1806    0.61%    7.31   2254   0.67%   5.18
8      Mississauga   1778      34%    0.23   2179   1.82%   4.21
9   North York. ON   1705    1.55%    1.07   2127  -0.42%   3.60
10          Ottawa   1689     4.39    6.76   2056   1.63%   2.85
11         Vaughan   1603    0.06%   -3.38   2142  -0.79%   8.40
12        Hamilton   1559    0.65%    6.93   1870   1.58%  -9.79
13         Markham   1556  -10.47%   -7.77   2107   8.55%   3.54
14        Brampton   1552

In [13]:
updfJan = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Jan2022-Jan2022.csv', 'January', '2022')
print(updfJan)

               City  1 BED     M/M     Y/Y   2 BED   M/M.1    Y/Y.1
0         Vancouver   2176   2.06%  13.22%    2983  -0.50%    9.75%
1           Toronto   2013  -1.42%   9.22%  $2,715  -1.95%   11.64%
2         Etobicoke   1857   2.94%   4.09%  $2,408   5.24%    8.66%
3       Mississauga   1772  -1.34%  -2.32%  $2,140  -0.09%    1.37%
4          York. ON   1706   1.73%  -1.78%  $2.250   4.17%    0.85%
5        North York   1679   1.27%  -2.21%  $2.136   0.75%    1.67%
6         Brantford   1648   3.13%  11.73%  $1.767  -0.62%    2.43%
7          Brampton   1621   4.45%   7.14%   $1884   0.48%    3.57%
8            Ottawa   1618   1.13%   2.21%  $2,023  -0.20%    1.30%
9              Ajax   1617   5.89%  -0.92%  $1.956   0.15%   -1.01%
10          Vaughan   1602  -3.78%  -4.98%  $2.159   4.86%     5.83
11          Halifax   1602  -3.03%  10.41%  $1,937  -0.82%   -2.52%
12        East York   1580  -1.50%  -1.99%  $2,127    0.76    5.30%
13         Victoria   1566  -7.61%  -0.63%  $2,4

In [14]:
updfJun = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/June2022-June2022.csv', 'June', '2022')
print(updfJun)

              City   1 BED    M/M     Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,377   1.8%   19.9%  $3,495    5.1%  24.1%
1          Toronto  $2,133   3.3%   15.7%  $3,002    5.4%  21.5%
2          Burnaby  $2,012   6.9%   24.1%  $2,645    3.2%  18.9%
3         Oakville  $2,011   0.0%    8.2%  $2,445    4.8%   8.9%
4       Burlington  $1,978  -1.9%   11.0%  $2.331    3.1%  13.3%
5        Etobicoke  $1,972   4.0%   14.0%  $2,586    5.7%  17.4%
6           Guelph  $1,950   7.7%   19.6%  $2.156    -11%  12 9%
7    Richmond Hill  $1,928   2.9%   20.3%  $2,296    5.3%    72%
8         Victoria  $1,870   5.3%   13.4%  $2,896    5.8%    NaN
9      Mississauga  $1,855   2.1%    5.6%  $2.282    3.2%  10.7%
10       Kitchener  $1,839   8.1%   22.4%  $2.179    7.0%  23.7%
11      North York  $1,781   3.2%     79%  $2,367     48%  14.5%
12         Vaughan  $1,720  -0.6%    6.0%  $2,226  -4. 8%   4.2%
13          London  $1.678    NaN     NaN  $2,023   19.6%  25.0%
14          Ottawa  $1,67

In [19]:
updfMar = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Mar2022-Mar2022.csv', 'March', '2022')
print(updfMar)

              City   1 BED     M/M     Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,239   3.51%  14.70%  $3,050   1.57%  17.62
1          Toronto  $2,044   0.39%  12.68%  $2,778  -0.71%  15.13
2          Burnaby  $1,979   7.15%  16.82%  $2,398   0.88%   7.63
3         Oakville  $1,922  -1.89%   9.77%  $2,421   1.38%  14.79
4        Etobicoke  $1,860   0.00%   6.41%  $2,452   0.91%  13.15
5           Oshawa  $1,822  -1.46%     NaN  $1,846   6.15%   4.53
6      Mississauga  $1,776  -0.11%   0.91%  $2.193   0.64%   6.61
7           Barrie  $1,771     NaN     NaN  $2,029   0.84%    NaN
8           Guelph  $1.729   6.20%   9.50%  $2,112   2.33%  10.34
9   North York. ON  $1.724   1.11%   1.89%   $2125  -0.09%   0.66
10          Ottawa  $1,692   0.18%   9.87%  $2.033  -1.12%   2.99
11         Vaughan  $1,671   4.24%   2.20%  $2,208   3.08%  11.52
12         Halifax  $1,663  11.61%  16.62%  $2,028    3.36   9.15
13        Victoria  $1.661   7.79%   3.17%  $2,667    5.33    NaN
14       K

In [20]:
updfNov = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Nov2021-Table 1.csv', 'November', '2021')
print(updfNov)

               City   1 BED    M/M    Y/Y  2 BED   M/M.1   Y/Y.1
0         Vancouver  $2,216   2.8%  16.6%   3057    4.3%   12.7%
1           Toronto  $2,006   3.5%   4.3%   2678     1.9    5.2%
2          Oakville  $1,984   0.3%   2.9%   2346    0.0%    4.2%
3        Burlington  $1,888   3.5%   2.1%   2244    1.6%    9.8%
4      Victoria. BC  $1,863   8.6%  19.4%   2432   -7.0%   18.2%
5           Vaughan  $1,828  -1.6%  11.6%   2077   -2.1%    1.3%
6       Mississauga  $1.824    0.9  -2.8%   2160   -0.9%    0.4%
7         Etobicoke  $1.769  -1.6%  -5.0%   2303    -2.4     -1%
8            Guelph  $1,766   3.5%  11.3%   2115    5.3%   10.2%
9           Burnaby  $1,759  -4.0%   6.7%   2280    -3.6    1.5%
10      Markham. ON  $1,724  -4.4%  -2.6%   2003   -7.2%   -0.9%
11       North York  $1.724   0.6%   -9.4   2111    1.2%   -3.7%
12          Halifax  $1,637   4.0%  17.5%   1928   -5.6%    3.6%
13           Ottawa  $1,620  -0.2%   1.8%   2028    -0.4    1.9%
14  New Westminster  $1,6

In [21]:
updfOct = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Oct2021-Table 1.csv', 'October', '2021')
print(updfOct)

                 City   1 BED    M/M     Y/Y   2 BED  M/M.1   Y/Y.1
0           Vancouver  $2,155  -0.5%   11.0%  $2,932  -3.7%    8.1%
1             Toronto  $1,938  -2.6%   -1.5%  $2,628   0.0%     21%
2             Vaughan  $1,858    0.4    7.4%  $2,122  -0.1%   -0.4%
3             Burnaby  $1,833   3.8%    8.5%  $2,366   1.0%    7.5%
4          Burlington  $1,823  -0.5%   -3.2%  $2.209   0.2%    9.9%
5         Mississauga  $1,809   1.1%   -6.4%  $2,180  -0.3%    1.2%
6             Markham  $1.803   2.5%    2.3%  $2.157   9.3%    3.7%
7           Etobicoke  $1.797  -1.1%   -4.9%  $2,360   4.0%    0.3%
8            Victoria  $1,715    NaN    6.1%  $2.614    NaN     NaN
9      North York. ON   $1714   1.9%   -5.7%  $2.086   1.8%   -4.4%
10             Guelph  $1,706  -1.3%   10.7%  $2.009  -1.3%    9.2%
11         Gloucester  $1.689   1.7%     NaN  $2,282   2.5%     NaN
12             Ottawa  $1,623   1.5%   -1.9%  $2,036   1.5%   -3.4%
13         Surrev. BC  $1,623   2.3%     NaN  $1

In [22]:
updfSep = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/Sep2021-Table 1.csv', 'September', '2021')
print(updfSep)

               City  1 BED    M/M     Y/Y  2 BED  M/M.1   Y/Y.1
0         Vancouver   2167  -0.8%    15.7   3044   0.1%   12.4%
1           Toronto   1989   7.2%   -1.2%   2628   0.8%   -1.3%
2           Vaughan   1850   8.9%    5.3%   2124   0.5%   -5.4%
3        Burlington   1832   5.0%   -4.9%   2205   6.8%   10.5%
4         Etobicoke   1816   0.8%   -6.2%   2269   -3.9   -7.8%
5       Mississauga   1790   2.1%   -7.2%   2187   2.7%    1.4%
6           Burnaby   1765   5.6%    5.1%   2341   4.3%    7.2%
7           Markham   1760  -1.4%   -4.7%   1974  -0.1%     NaN
8            Guelph   1728   0.8%   13.4%   2035   1.8%   16.1%
9        North York   1681   0.8%  -10.0%   2049   1.2%   -8.7%
10  New Westminster   1623  -2.7%    5.7%   1984  -1.3%   -7.7%
11      Scarborough   1605   1.2%   -5.0%   1906   2.9%   -4.7%
12           Ottawa   1598  -0.8%   -1.8%   2005  -1.4%   -5.6%
13           Surrey   1586   2.6%     NaN   1756   0.2%     NaN
14           Nepean   1576   1.7%    0.9